In [2]:
import plotly.graph_objs as go
import pandas as pd
import requests

In [3]:
reports_r = requests.get('https://peridash.ml/api/reports')
measurements_r = requests.get('https://peridash.ml/api/measurements')
epsas_r = requests.get('https://peridash.ml/api/epsas')
variables_r = requests.get('https://peridash.ml/api/variables')
indicators_r = requests.get('https://peridash.ml/api/indicators')

In [4]:
reports_df = pd.read_json(reports_r.text)
measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)
variables_df = pd.read_json(variables_r.text)
indicators_df = pd.read_json(indicators_r.text)

complete_reports_df = pd.merge(reports_df, epsas_df, left_on='epsa', right_on='url')
complete_measurements_df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

In [5]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [6]:
ind_names = [
    'Rendimiento actual de la fuente', 'Uso eficiente del recurso',
    'Cobertura de muestras de agua potable',
    'Conformidad de los análisis de agua potable realizados',
    'Dotación', 'Continuidad por racionamiento', 'Continuidad por corte',
    'Cobertura del servicio de agua potable',
    'Cobertura del servicio de alcantarillado sanitario',
    'Cobertura de micromedición',
    'Incidencia extracción de agua cruda subterránea ',
    'Índice de tratamiento de agua residual', 'Control de agua residual',
    'Capacidad instalada de planta de tratamiento de agua potable',
    'Capacidad instalada de planta de tratamiento de agua residual ',
    'Presión del servicio de agua potable',
    'Índice de agua no contabilizada en producción',
    'Índice de agua no contabilizada en la red',
    'Densidad de fallas en tuberías de agua potable',
    'Densidad de fallas en conexiones de agua potable',
    'Densidad de fallas en tuberías de agua residual',
    'Densidad de fallas en conexiones de agua residual',
    'Índice de operación eficiente', 'Prueba ácida',
    'Eficiencia de recaudación', 'Índice de endeudamiento total', 'Tarifa media',
    'Costo unitario de operación', 'Índice de ejecución de inversiones',
    'Personal calificado', 'Número de empleados por cada 1000 conexiones',
    'Atención de reclamos'
]
ind_units = ['%', '%', '%', '%', 'l/hab/día', 'hr/día', '%', '%', '%', '%',
             '%', '%', '%', '%', '%', '%', '%', '%', 'fallas/100km',
             'fallas/1000conex.', 'fallas/100km', 'fallas/1000conex.', '%', '-',
             '%', '%', '%CUO(Bs.)', '%TM(Bs.)', '%', '%', 'empleados/1000conex.', '%']

colors = [
    '#1f77b4','#ff7f0e','#2ca02c',
    '#d62728', '#9467bd', '#8c564b',
    '#e377c2', '#7f7f7f', '#bcbd22','#17becf'
]

def get_ind_name(ind):
    return ind_names[int(ind[3:])-1]

def get_ind_unit(ind):
    return ind_units[int(ind[3:])-1]

In [7]:
crdf = complete_reports_df

category = 'A'
state = 'SC'

variables = ['v1', 'v2', 'v3']
states = ['LP', 'SC', 'CO', 'PO', 'BE', 'TA', 'CH', 'PA', 'OR']

state_name_map = {
    'LP': 'La Paz',
    'SC': 'Santa Cruz',
    'CO': 'Cochabamba',
    'PO': 'Potosí',
    'BE': 'Beni',
    'TA': 'Tarija',
    'CH': 'Chuquisaca',
    'PA': 'Pando',
    'OR': 'Oruro', 
}

def get_domain(var):
    if var == 'v1':
        return {"x": [0, .30]}
    if var == 'v2':
        return {"x": [.32, .62]}
    else:
        return {"x": [.64, .94]}
        
def get_visible_list(state):
    state_map = {'LP': 0, 'SC': 1, 'CO': 2, 
                 'PO': 3, 'BE': 4, 'TA': 5,
                 'CH': 5, 'PA': 7, 'OR': 8}
    
    offset = state_map[state]
    
    base_array = [0 for i in range(9)]
    base_array[offset] = 1
    
    big_true = [True for i in range(3)]
    big_false = [False for i in range(3)]
    
    nested_array = [big_true if x == 1 else big_false for x in base_array]
    
    return [x for l in nested_array for x in l]

data = []

for state in states:
    filtered_df = crdf[(crdf.state == state) & (crdf.year == 2017)]
    visible = True if state == 'LP' else False
    for var in variables:
        data.append(dict(
            values=list(filtered_df[var]),
            labels=list(filtered_df.code),
            type='pie',
            hole=.4,
            textposition='inside',
            name=var,
            domain=get_domain(var),
            hoverinfo='label+value+percent+name',
            visible=visible,
        ))


var1 = variables_df[variables_df.var_id == 1]
var2 = variables_df[variables_df.var_id == 2]
var3 = variables_df[variables_df.var_id == 3]

v1name = var1.name.iloc[0]
v2name = var2.name.iloc[0]
v3name = var3.name.iloc[0]

v1unit = var1.unit.iloc[0]
v2unit = var2.unit.iloc[0]
v3unit = var3.unit.iloc[0]

updatemenus = [dict(
    active=0,
    xanchor='left',
    yanchor='top',
    direction='up',
    x= .4,
    y= -0.05,
    buttons = [dict(
        label=f'{state_name_map[state]}',
        method='update', # modify both data and layout
        args = [
            {'visible': get_visible_list(state)}, # data modification
            # layout modification
            dict(
#                 title= make_title(ind),
#                 xaxis= make_x_axis(cat),
#                 yaxis= make_y_axis(ind),
#                 shapes= make_shapes(ind),
            ),
        ],
    ) for state in states]
)]


layout = dict(
    updatemenus = updatemenus,
    title = f'<b>V1</b>: {v1name}({v1unit})<br><b>V2</b>: {v2name}({v2unit})<br><b>V3</b>: {v3name}({v3unit})',
    titlefont = dict(size=15),
    annotations = [dict(
        font=dict(size=20),
        showarrow=False,
        text='V1',
        x=0.13,
        y=0.5,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='V2',
        x=0.47,
        y=0.5,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='V3',
        x=0.81,
        y=0.5,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='Fuentes<br>Superficiales',
        x=0.07,
        y=0,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='Fuentes<br>Subterráneas',
        x=0.48,
        y=0,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='Volumen AP<br>Producido',
        x=0.87,
        y=0,
    )]
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [8]:
import plotly.plotly as py

py.iplot(fig, filename='volumen1_2017')

In [9]:
crdf = complete_reports_df

category = 'A'
state = 'SC'

variables = ['v4', 'v5', 'v6']
states = ['LP', 'SC', 'CO', 'PO', 'BE', 'TA', 'CH', 'PA', 'OR']

state_name_map = {
    'LP': 'La Paz',
    'SC': 'Santa Cruz',
    'CO': 'Cochabamba',
    'PO': 'Potosí',
    'BE': 'Beni',
    'TA': 'Tarija',
    'CH': 'Chuquisaca',
    'PA': 'Pando',
    'OR': 'Oruro', 
}

def get_domain(var):
    if var == 'v4':
        return {"x": [0, .30]}
    if var == 'v5':
        return {"x": [.32, .62]}
    else:
        return {"x": [.64, .94]}
        
def get_visible_list(state):
    state_map = {'LP': 0, 'SC': 1, 'CO': 2, 
                 'PO': 3, 'BE': 4, 'TA': 5,
                 'CH': 5, 'PA': 7, 'OR': 8}
    
    offset = state_map[state]
    
    base_array = [0 for i in range(9)]
    base_array[offset] = 1
    
    big_true = [True for i in range(3)]
    big_false = [False for i in range(3)]
    
    nested_array = [big_true if x == 1 else big_false for x in base_array]
    
    return [x for l in nested_array for x in l]

data = []

for state in states:
    filtered_df = crdf[(crdf.state == state) & (crdf.year == 2017)]
    visible = True if state == 'LP' else False
    for var in variables:
        data.append(dict(
            values=list(filtered_df[var]),
            labels=list(filtered_df.code),
            type='pie',
            hole=.4,
            textposition='inside',
            name=var,
            domain=get_domain(var),
            hoverinfo='label+value+percent+name',
            visible=visible,
        ))


var4 = variables_df[variables_df.var_id == 4]
var5 = variables_df[variables_df.var_id == 5]
var6 = variables_df[variables_df.var_id == 6]

v4name = var4.name.iloc[0]
v5name = var5.name.iloc[0]
v6name = var6.name.iloc[0]

v4unit = var4.unit.iloc[0]
v5unit = var5.unit.iloc[0]
v6unit = var6.unit.iloc[0]

updatemenus = [dict(
    active=0,
    xanchor='left',
    yanchor='top',
    direction='up',
    x= .4,
    y= -0.05,
    buttons = [dict(
        label=f'{state_name_map[state]}',
        method='update', # modify both data and layout
        args = [
            {'visible': get_visible_list(state)}, # data modification
            # layout modification
            dict(
#                 title= make_title(ind),
#                 xaxis= make_x_axis(cat),
#                 yaxis= make_y_axis(ind),
#                 shapes= make_shapes(ind),
            ),
        ],
    ) for state in states]
)]


layout = dict(
    updatemenus = updatemenus,
    title = f'<b>V4</b>: {v4name}({v4unit})<br><b>V5</b>: {v5name}({v5unit})<br><b>V6</b>: {v6name}({v6unit})',
    titlefont = dict(size=15),
    annotations = [dict(
        font=dict(size=20),
        showarrow=False,
        text='V4',
        x=0.13,
        y=0.5,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='V5',
        x=0.47,
        y=0.5,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='V6',
        x=0.81,
        y=0.5,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='AP en planta de<br>tratamiento',
        x=0.07,
        y=0,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='AP<br>Facturado',
        x=0.48,
        y=0,
    ), dict(
        font=dict(size=20),
        showarrow=False,
        text='Agua<br>Residual',
        x=0.87,
        y=0,
    )]
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [10]:
py.iplot(fig, filename='volumen2_2017')